In [5]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git 

In [6]:

import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Activation, Conv2D, Dropout, LeakyReLU, Input, Concatenate, Conv2DTranspose
from tensorflow.keras.models import Sequential, Model,Input
from tensorflow.keras.initializers import RandomNormal
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization 

In [20]:
def discriminatorModel(imageShape):
    # weight initialization
    model = Sequential()
    model.add(Input(shape=imageShape))
    model.add(Conv2D(64, (4, 4), strides=(2,2), padding='same',input_shape=imageShape))
    model.add(InstanceNormalization(axis=-1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (4, 4), strides=(2,2), padding='same'))
    model.add(InstanceNormalization(axis=-1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(256, (4, 4), strides=(2,2), padding='same'))
    model.add(InstanceNormalization(axis=-1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(512, (4, 4), strides=(2,2), padding='same'))
    model.add(InstanceNormalization(axis=-1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(512, (4, 4), strides=(1,1), padding='same'))
    model.add(InstanceNormalization(axis=-1))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.5))
    model.add(Conv2D(1, (4,4)))
    # We need to slow down the rate at which the descriminator learns
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'],loss_weights=[0.5])
    return model
model = discriminatorModel((256,256,3))    
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_65 (Conv2D)           (None, 128, 128, 64)      3136      
_________________________________________________________________
instance_normalization_55 (I (None, 128, 128, 64)      128       
_________________________________________________________________
leaky_re_lu_55 (LeakyReLU)   (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 64, 64, 128)       131200    
_________________________________________________________________
instance_normalization_56 (I (None, 64, 64, 128)       256       
_________________________________________________________________
leaky_re_lu_56 (LeakyReLU)   (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 32, 32, 256)     

In [40]:
def residualBlock(n_filters, input_layer):
    init = RandomNormal(stddev=0.02)
    resTensor = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(input_layer)
    resTensor =InstanceNormalization(axis=-1)(resTensor)
    resTensor = Activation('relu')(resTensor)
    resTensor = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(resTensor)
    resTensor =InstanceNormalization(axis=-1)(resTensor)
    resTensor = Concatenate()([resTensor,input_layer])
    return resTensor

In [43]:
def generatorModel(LImageShape, LResnet=9):
    # init = RandomNormal(stddev=0.02)
    
    in_shape = Input(shape=LImageShape)
    genTensor = Conv2D(64, (7,7), padding = 'same',)(in_shape)
    genTensor = InstanceNormalization(axis=-1)(genTensor)
    genTensor = Activation('relu')(genTensor)
    genTensor = Conv2D(128, (3,3), strides = (2,2), padding = 'same')(genTensor)
    genTensor = InstanceNormalization(axis=-1)(genTensor)
    genTensor = Activation('relu')(genTensor)
    genTensor = Conv2D(258, (3,3), strides = (2,2), padding = 'same')(genTensor)
    genTensor = InstanceNormalization(axis=-1)(genTensor)
    genTensor = Activation('relu')(genTensor)
    
    for _ in range(LResnet):
        genTensor = residualBlock(256, genTensor)
    
    genTensor = Conv2DTranspose(128, (3,3), strides = (2,2), padding = 'same')(genTensor)
    genTensor = InstanceNormalization(axis=-1)(genTensor)
    genTensor = Conv2DTranspose(64, (3,3), strides = (2,2), padding = 'same')(genTensor)
    genTensor = InstanceNormalization(axis=-1)(genTensor)
    genTensor = Conv2D(3, (7,7), padding='same', activation='tanh',)(genTensor)
    genTensor = InstanceNormalization(axis=-1)(genTensor)
    
    #TO-DO: This part
    
    out_image = Activation('tanh')(genTensor)
    
    # define model
    model = Model(in_shape, out_image)
    return model

model = generatorModel((256,256,3)) 
model.summary()  

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_159 (Conv2D)             (None, 256, 256, 64) 9472        input_24[0][0]                   
__________________________________________________________________________________________________
instance_normalization_147 (Ins (None, 256, 256, 64) 128         conv2d_159[0][0]                 
__________________________________________________________________________________________________
activation_46 (Activation)      (None, 256, 256, 64) 0           instance_normalization_147[0][0] 
______________________________________________________________________________________________

In [143]:
def define_composite_model(g_model_1, d_model, g_model_2, image_shape):
    g_model_1.trainable = True
    d_model.trainable = False
    g_model_2.trainable = False

    
    #TO-DO: ?
    
    input_gen = Input(shape=image_shape)
    gen1_out = g_model_1(input_gen)
    output_d = d_model(gen1_out)

    input_id = Input(shape=image_shape)
    output_id = g_model_1(input_id)

    output_f = g_model_2(gen1_out)
    gen2_out = g_model_2(input_id)
    output_b = g_model_1(gen2_out)

    model = Model([input_gen, input_id], [output_d, output_id, output_f, output_b])
    model.compile(loss=['mse', 'mae', 'mae', 'mae'], loss_weights=[1, 5, 10, 10], optimizer='adam')
    return model

In [144]:
def load_real_samples(filename):
    data = np.load(filename)
    X1, X2 = data['arr_0'], data['arr_1']
    X1 = (X1 - 127.5) / 127.5
    X2 = (X2 - 127.5) / 127.5
    return [X1, X2]

In [145]:
#TO-DO: Upload Images
image_shape = (256,256,3,1)
g_model_AtoB = generatorModel(image_shape)